In [ ]:
pip install kagglehub[pandas-datasets]

In [ ]:
import tensorflow as tf
import kagglehub
import os
from tensorflow.data import AUTOTUNE

In [ ]:
train_dir = kagglehub.dataset_download("jessicali9530/celeba-dataset")  # Download dataset
image_dir = os.path.join(train_dir, "img_align_celeba")  # Path to images

100%|█████████████████████████████████████████████████████████████████████████████| 1.33G/1.33G [01:56<00:00, 12.3MB/s]

Extracting files...


### Ignore Until here

In [ ]:
pip install --user kagglehub

In [ ]:
pip install --user pyxlsb

In [1]:
import tensorflow as tf
import kagglehub
import os
import pandas as pd
from tensorflow.data import AUTOTUNE
import numpy as np
from sklearn.preprocessing import LabelEncoder

2025-09-29 19:52:32.742736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759189952.936079  516823 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759189952.984839  516823 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759189953.507345  516823 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759189953.507395  516823 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759189953.507399  516823 computation_placer.cc:177] computation placer alr

In [2]:
attributes_df = pd.read_csv("identity_CelebA.txt", delim_whitespace=True, names=['image_id','Celeb_identity'], header=None)

In [3]:
print(f"Original dataset size: {len(attributes_df)}")
attributes_df = attributes_df.dropna(subset=['image_id', 'Celeb_identity'])
print(f"After removing missing values: {len(attributes_df)}")


Original dataset size: 202599
After removing missing values: 202599


In [4]:
attributes_df['Celeb_identity'] = attributes_df['Celeb_identity'].astype(int)


In [5]:
print(f"Dataset Shape: {attributes_df.shape}")
print(f"Columns: {list(attributes_df.columns)}")
print(f"Number of unique identities: {attributes_df['Celeb_identity'].nunique()}")

Dataset Shape: (202599, 2)
Columns: ['image_id', 'Celeb_identity']
Number of unique identities: 10177


In [6]:
train_dir = kagglehub.dataset_download("jessicali9530/celeba-dataset")  # Download dataset
image_dir = os.path.join(train_dir, "img_align_celeba", "img_align_celeba")  # Path to images

In [7]:
# Prepare image paths and labels
image_paths = [os.path.join(image_dir, img_name) for img_name in attributes_df['image_id']]

# Encode identity labels to integers (0 to num_classes-1)
label_encoder = LabelEncoder()
identity_labels = label_encoder.fit_transform(attributes_df['Celeb_identity'])
num_classes = len(label_encoder.classes_)

print(f"Total images: {len(image_paths)}")
print(f"Number of classes (identities): {num_classes}")
print(f"Original identity range: {attributes_df['Celeb_identity'].min()} to {attributes_df['Celeb_identity'].max()}")
print(f"Encoded label range: {identity_labels.min()} to {identity_labels.max()}")


Total images: 202599
Number of classes (identities): 10177
Original identity range: 1 to 10177
Encoded label range: 0 to 10176


In [8]:
# to load and preprocess images
def load_and_preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [128, 128])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((image_paths, identity_labels))

# Apply preprocessing
dataset = dataset.map(
    load_and_preprocess_image,
    num_parallel_calls=AUTOTUNE
)

# Split into train/validation (80/20 split)
total_size = len(image_paths)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

# Shuffle and split
dataset = dataset.shuffle(buffer_size=10000, seed=42)
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Batch and prefetch for training
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

print(f"Training batches: {len(train_dataset)}")
print(f"Validation batches: {len(val_dataset)}")

# Optional: Cache datasets to speed up training (if you have enough memory/storage)
# train_dataset = train_dataset.cache()
# val_dataset = val_dataset.cache()

# Verify the dataset
for images, labels in train_dataset.take(1):
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    print(f"Label range: {tf.reduce_min(labels)} to {tf.reduce_max(labels)}")
    break

2025-09-29 19:52:50.255127: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Training batches: 5065
Validation batches: 1267
Image batch shape: (32, 128, 128, 3)
Label batch shape: (32,)
Label range: 39 to 10031


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

def create_baseline_cnn(num_classes, input_shape=(128, 128, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.GlobalAveragePooling2D(),

        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),

        layers.Dense(num_classes, activation='softmax')
    ])

    return model



In [10]:
# Create the model
print("Creating baseline CNN model...")
baseline_model = create_baseline_cnn(num_classes)
print("Model Architecture:")
baseline_model.summary()

Creating baseline CNN model...
Model Architecture:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10177)          │     2,615,489 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,268,737 (12.47 MB)

 Trainable params: 3,267,777 (12.47 MB)

 Non-trainable params: 960 (3.75 KB)

In [11]:
# Compile the model
baseline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Model compiled successfully!")
print(f"Total parameters: {baseline_model.count_params():,}")



Model compiled successfully!
Total parameters: 3,268,737


In [9]:
# Training configuration
EPOCHS = 20
EARLY_STOPPING_PATIENCE = 5

# Callbacks for better training
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True,
        verbose=1
    ),
    ### tf.keras.callbacks.ReduceLROnPlateau(
        #monitor='val_loss',
        #factor=0.5,
       # patience=3,
      #  min_lr=1e-7,
     #   verbose=1
    #),
    #tf.keras.callbacks.ModelCheckpoint(
        #filepath='baseline_cnn_best.h5',
       # monitor='val_accuracy',
      #  save_best_only=True,
     #   verbose=1
    #)
]
# Start training
# history = baseline_model.fit(
#     train_dataset,
#     epochs=EPOCHS,
#     validation_data=val_dataset,
#     callbacks=callbacks,
#     verbose=1
# )

# print(f"\nTraining completed!")
# print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
# print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels=None,
    label_mode=None,
    image_size=(128, 128),
    batch_size=32,
    shuffle=True
)

# Normalize the images to [0, 1] range
def normalize(image):
    return tf.cast(image, tf.float32) / 255.0

# Apply the normalization
train_dataset = train_dataset.map(normalize, num_parallel_calls=AUTOTUNE)

# Prefetch to optimize dataset loading
train_dataset = train_dataset.prefetch(AUTOTUNE)

In [11]:
# Count frequency of each identity
top_ids = attributes_df['Celeb_identity'].value_counts().nlargest(1000).index.tolist()

# Filter dataframe to only those identities
filtered_df = attributes_df[attributes_df['Celeb_identity'].isin(top_ids)].copy()


In [12]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
filtered_df['encoded_label'] = label_encoder.fit_transform(filtered_df['Celeb_identity'])

# Update image paths
image_paths = [os.path.join(image_dir, img_name) for img_name in filtered_df['image_id']]
identity_labels = filtered_df['encoded_label'].values
NUM_CLASSES = len(label_encoder.classes_)  # should be 1000


In [13]:
# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((image_paths, identity_labels))

# Preprocessing
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

# Shuffle & Split (80/20)
total_size = len(image_paths)
train_size = int(0.8 * total_size)

dataset = dataset.shuffle(buffer_size=10000, seed=42)
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Batch and prefetch
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import time

# ----------- Residual Block ------------
def residual_block(x, filters, stride=1):
    shortcut = x

    # First conv
    x = layers.Conv2D(filters, (3, 3), strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second conv
    x = layers.Conv2D(filters, (3, 3), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Adjust shortcut if dimensions change
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x


In [15]:
def build_resnet_style_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Initial conv
    x = layers.Conv2D(64, (7, 7), strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 512, stride=2)

    # Classification head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs, name="ResNet_Style")
    return model


In [ ]:
# Example config
INPUT_SHAPE = (128, 128, 3)
NUM_CLASSES = 1000  # <-- update for your dataset
BATCH_SIZE = 32
EPOCHS = 20

# Assume X_train, y_train_cat, X_val, y_val_cat are ready
resnet_model = build_resnet_style_model(INPUT_SHAPE, NUM_CLASSES)
resnet_model.compile(
    optimizer=optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Training
history_resnet = resnet_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)



Epoch 1/20
 99/751 ━━━━━━━━━━━━━━━━━━━━ 8:12 755ms/step - accuracy: 0.0028 - loss: 7.0064

In [18]:
val_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
_# Evaluation
resnet_test_loss, resnet_test_acc, resnet_test_top5 = resnet_model.evaluate(X_test, y_test_cat, verbose=0)

results = {}
results['ResNet_Style'] = {
    'test_acc': resnet_test_acc,
    'best_val_acc': max(history_resnet.history['val_accuracy'])
}

print(f"ResNet-style Test Accuracy: {resnet_test_acc*100:.2f}%")
